In [2]:
import subprocess
from pydub import AudioSegment # type: ignore
import math
import openai # type: ignore
import glob

# 비디오에서 오디오 추출
def extract_audio_from_video(video_path, autio_path):
    command = [
        "ffmpeg", 
        "-i",           # 인풋
        video_path, 
        "-vn",          # 비디오 없이
        autio_path,
    ]

    subprocess.run(command)

# 오디오에서 10분 단위로 자르기
def cut_audio_in_chunk(audio_path, chunk_size, chunk_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunk_folder}/chunk_{i}.mp3", format="mp3")



extract_audio_from_video("./files/podcast.mp4", "./files/podcast.mp3",) # 영상파일을 음성파일로
cut_audio_in_chunk("./files/podcast.mp3", 10, "./files/chunks")
# track.duration_seconds




In [5]:


# transcript = openai.Audio.transcribe("whisper-1", open("./files/chunks/chunk_0.mp3", "rb"), )
# transcript["text"]

# 오디오 파일에서 텍스트로 추출하기
def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe(
                "whisper-1", 
                audio_file, 
            )
            text_file.write(transcript["text"])
    

transcribe_chunks(
    "./files/chunks", 
    "./files/transcript.txt",
)


# transcript = openai.audio.transcriptions.create(
# model="whisper-1",
# file=open("./files/chunks/chunk_0.mp3", "rb"),
# language="en"
# )

